In [1]:
import pandas as pd

In [2]:
path = "https://raw.githubusercontent.com/yurchisin/disaster_prepositioning/refs/heads/main/content/data/"

simple_Allocation = pd.read_csv(path+"simple_Allocation.csv")
simple_Allocation

,depotCity,drivingTime_hrs,Buckets
0,Ambanja,14.00,375
1,Ambatondrazaka,0.00,26
2,Ambositra,11.00,41
3,Ambovombe,26.00,2322
4,Antananarivo Renivohitra,6.00,9046
5,Antsohihy,10.00,610
6,Farafangana,19.00,5201
7,Fenerive Est,11.00,6689
8,Mahajanga I,16.00,150
9,Maintirano,14.25,2460


In [3]:
import gurobipy as gp
from gurobipy import GRB

# Prep data
BucketsNeeded = 13561
n = len(simple_Allocation)
t = simple_Allocation.drivingTime_hrs
b = simple_Allocation.Buckets

# Create model
model = gp.Model("simple_Allocation") 

# Add decision variables
y=model.addVars(n, name="Warehouse_Allocation")

# Add constraint to meet demand
model.addConstr(gp.quicksum(y[i] for i in range(n))==BucketsNeeded,name='Meet_Demand')

# Add in warehouse_constraints
for i in range(n):
    model.addConstr(y[i] <= b[i], name=f"warehouse_endowment_{i}")

# Note we don't have a constraint for y >= 0 since it's assumed in the variable definition
# Add objective
objective = gp.quicksum(t[i] * y[i] for i in range(n))
model.setObjective(objective, GRB.MINIMIZE)

# Fire up the solver!
model.optimize()

Restricted license - for non-production use only - expires 2025-11-24


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 24.4.0 24E263)


CPU model: Apple M1


Thread count: 8 physical cores, 8 logical processors, using up to 8 threads


Optimize a model with 17 rows, 16 columns and 32 nonzeros


...

Iteration    Objective       Primal Inf.    Dual Inf.      Time


       0    0.0000000e+00   1.691875e+03   0.000000e+00      0s


       1    9.8293000e+04   0.000000e+00   0.000000e+00      0s


Solved in 1 iterations and 0.01 seconds (0.00 work units)


Optimal objective  9.829300000e+04


Now Let's Analyze the results!

In [4]:
y_sol = []
for i in range(n):
    y_sol.append(y[i].X)
simple_Allocation_sol = simple_Allocation.copy()
simple_Allocation_sol['y'] = y_sol
simple_Allocation_sol.sort_values('drivingTime_hrs')

,depotCity,drivingTime_hrs,Buckets,y
1,Ambatondrazaka,0.00,26,26.0
4,Antananarivo Renivohitra,6.00,9046,9046.0
11,Miarinarivo,7.00,3,3.0
14,Toamasina I,8.00,1580,1580.0
5,Antsohihy,10.00,610,610.0
2,Ambositra,11.00,41,0.0
7,Fenerive Est,11.00,6689,2296.0
0,Ambanja,14.00,375,0.0
9,Maintirano,14.25,2460,0.0
8,Mahajanga I,16.00,150,0.0


### Second ESUPS example model

In [5]:
warehouses = pd.read_csv(path+"warehouses.csv")
warehouses

,Type,Lat,Long,Buckets
0,warehouses,-17.823700,48.426300,26
1,warehouses,-20.516700,47.250000,41
2,warehouses,-18.908500,47.537500,9046
3,warehouses,-17.384300,49.409800,2762
4,warehouses,-16.916700,49.900000,1682
5,warehouses,-15.430900,49.758300,1870
6,warehouses,-16.170200,49.774100,0
7,warehouses,-16.926100,49.587100,375
8,warehouses,-25.176133,46.089378,2322
9,warehouses,-25.031600,46.990000,0


In [6]:
disasters = pd.read_csv(path+"disasters.csv")
total_buckets = warehouses['Buckets'].sum()

# one bucket is needed for 2.5 people on average
disasters['demand'] = (
    (disasters['People Impacted'] / 2.5)
      .round()
      .clip(upper=total_buckets)
      .astype(int)
)

disasters

,Type,Lat,Long,People Impacted,demand
0,Storm,-12.266700,49.283300,118000,40811
1,Storm,-14.266700,50.166700,100215,40086
2,Epidemic,-14.876200,47.983500,21976,8790
3,Epidemic,-15.716700,46.316700,15172,6069
4,Flood,-16.950400,46.828100,20000,8000
5,Flood,-17.384300,49.409800,28223,11289
6,Storm,-17.823700,48.426300,84309,33724
7,Storm,-18.064600,44.029500,526200,40811
8,Storm,-18.149900,49.402300,600000,40811
9,Epidemic,-18.769800,46.050000,3055,1222


In [7]:
distanceMatrix_scenarios = pd.read_csv(path+"distanceMatrix_scenarios.csv")
distanceMatrix_scenarios

,scenario,warehouse,drivingTime_hrs
0,0,0,21.00
1,0,1,30.00
2,0,2,25.00
3,0,3,32.00
4,0,4,35.75
...,...,...,...
457,21,16,26.00
458,21,17,19.00
459,21,18,48.00
460,21,19,33.00


In [8]:
demand = disasters['demand'].values
probs = [1/len(demand)] * len(demand)
demand

array([40811, 40086,  8790,  6069,  8000, 11289, 33724, 40811, 40811,
        1222, 22138,   760,  9348,  5424,   200, 40811, 40811, 40811,
       40000, 28000, 40811, 40811])

In [9]:
b

0      375
1       26
2       41
3     2322
4     9046
5      610
6     5201
7     6689
8      150
9     2460
10    4235
11       3
12     736
13    5700
14    1580
15    1637
Name: Buckets, dtype: int64

In [10]:
m = len(demand) #number of disasters/scenarios
t = distanceMatrix_scenarios.set_index(['scenario', 'warehouse']).squeeze().to_dict()
b = warehouses['Buckets'][warehouses['Buckets'] > 0].reset_index(drop=True)
n = len(b) #number of warehouses

model_2 = gp.Model("multiple_disasters") 

# Amount to take per Warehouse
y = model_2.addVars(t.keys(), name="Warehouse_Allocation")

# Add constraints to meet demand for each disaster scenario (k)
for k in range(m):
    # Demand constraints
    model_2.addConstr(gp.quicksum(y[k, i] for i in range(n)) == demand[k], name=f"Meet_Demand_K:{k}")
    
    # Warehouse constraints
    for i in range(n):
        model_2.addConstr(y[k, i] <= b[i], name=f"warehouse_endowment_K:{k}_I:{i}")

# Objective function to minimize the weighted driving time using t as a parameter
objective = gp.quicksum(
    probs[k] * gp.quicksum(t[k, i] * y[k, i] for i in range(n))
    for k in range(m)
)

# Optimize model
model_2.setObjective(objective, GRB.MINIMIZE)
model_2.optimize()

# Store results in the list 'a'
a = []
for v in model_2.getVars():
    a.append([v.VarName, v.X])
a = pd.DataFrame(a)
a.columns = ['Warehouse','Buckets']

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 24.4.0 24E263)


CPU model: Apple M1


Thread count: 8 physical cores, 8 logical processors, using up to 8 threads


Optimize a model with 484 rows, 462 columns and 924 nonzeros


Model fingerprint: 0xa84de061


...

Iteration    Objective       Primal Inf.    Dual Inf.      Time


       0    4.1096854e+05   1.040175e+04   0.000000e+00      0s


       6    4.4697821e+05   0.000000e+00   0.000000e+00      0s


Solved in 6 iterations and 0.00 seconds (0.00 work units)


Optimal objective  4.469782127e+05


In [11]:
a[:15]

,Warehouse,Buckets
0,"Warehouse_Allocation[0,0]",26.0
1,"Warehouse_Allocation[0,1]",41.0
2,"Warehouse_Allocation[0,2]",9046.0
3,"Warehouse_Allocation[0,3]",2762.0
4,"Warehouse_Allocation[0,4]",1682.0
5,"Warehouse_Allocation[0,5]",1870.0
6,"Warehouse_Allocation[0,6]",375.0
7,"Warehouse_Allocation[0,7]",2322.0
8,"Warehouse_Allocation[0,8]",610.0
9,"Warehouse_Allocation[0,9]",1027.0


In [12]:
a.to_csv("a.csv",index = False)

### Full Model

While this may initially look intimidating, it is one of the easiest changes to make to our current code. All we are doing is making a decision variable instead of a constant and then constraining it. Let’s substitute back in our equation from the last section with the new constraints to see this firsthand. To denote a new decision variable (i.e. a variable that can be changed), all we need to do is add it under the minimization sign. This means minimizing with respect to $x$ and $y$

$$
\min_{x,y} \sum_k P^k \sum_i \tau_{ij}\cdot y^k_i
$$

Then all we need to do is update the constraints. I've included the line to make it easier to see what's new as our list grows. It has no mathematical significance. 

So how do we Implement this in Gurobi?

$$
\begin{aligned}


\text{s.t.}  & \sum_{i} & y^k_{i}&=d^k & & \hspace{.2cm} \text{(total supplies sent must meet demand)}\\

& & y^k_i       &\leq x_i & \forall i \in I& \hspace{.2cm}\text{(you can't send more than a warehouse has)}\\

 &\text{} & y^k_{i} &\geq 0 &\forall i \in I& \hspace{.2cm} \text{(you can't send negative supplies)}\\
\hline \\
  & \sum_{i} & x_i&=\chi & & \hspace{.2cm} \text{(we allocate all supplies and no more)}\\

 && x_{i} &\geq 0 &\forall i \in I& \hspace{.2cm} \text{(you can't allocate negative supplies)}\\


\end{aligned}
$$

In [13]:
m = len(demand) #number of disasters/scenarios
t = distanceMatrix_scenarios.set_index(['scenario', 'warehouse']).squeeze().to_dict()
b = warehouses['Buckets'][warehouses['Buckets'] > 0].reset_index(drop=True)
n = len(b) #number of warehouses

model_full = gp.Model("full_allocation")

# Amount to take per warehouse
y = model_full.addVars(m, n, name="Single_Warehouse_Allocation")

# National allocation
x = model_full.addVars(n, name="National_Allocation")

# Total national endowment constraint
model_full.addConstr(gp.quicksum(x[i] for i in range(n)) == b.sum(), name="Total_National_Endowment")

# Add constraints to meet demand for each disaster scenario (k)
for k in range(m):
    model_full.addConstr(gp.quicksum(y[k, i] for i in range(n)) == demand[k], name=f"Meet_Demand_K:{k}")
    # Cannot allocate more than amount assigned to warehouse
    # x is now a decision variable
    for i in range(n):
        model_full.addConstr(y[k, i] <= x[i], name=f"warehouse_endowment_K:{k}_I:{i}")

# Objective function to minimize the weighted driving time using t as a parameter
objective = gp.quicksum(
    probs[k] * gp.quicksum(t[k, i] * y[k, i] for i in range(n))
    for k in range(m)
)

# Optimize model
model_full.setObjective(objective, GRB.MINIMIZE)
model_full.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 24.4.0 24E263)


CPU model: Apple M1


Thread count: 8 physical cores, 8 logical processors, using up to 8 threads


Optimize a model with 485 rows, 483 columns and 1407 nonzeros


Model fingerprint: 0x8817cae8


...

Iteration    Objective       Primal Inf.    Dual Inf.      Time


       0    3.4281409e+04   5.415380e+05   0.000000e+00      0s


     272    3.4027302e+05   0.000000e+00   0.000000e+00      0s


Solved in 272 iterations and 0.01 seconds (0.00 work units)


Optimal objective  3.402730155e+05


In [14]:
y_values_full = []
for i in range(m):
    for j in range(n):
        y_values_full.append([y[(i,j)].VarName, y[(i,j)].X])
y_values_full = pd.DataFrame(y_values_full, columns=['Warehouse', 'Buckets'])
y_values_full

,Warehouse,Buckets
0,"Single_Warehouse_Allocation[0,0]",0.0
1,"Single_Warehouse_Allocation[0,1]",18673.0
2,"Single_Warehouse_Allocation[0,2]",0.0
3,"Single_Warehouse_Allocation[0,3]",0.0
4,"Single_Warehouse_Allocation[0,4]",0.0
...,...,...
457,"Single_Warehouse_Allocation[21,16]",0.0
458,"Single_Warehouse_Allocation[21,17]",0.0
459,"Single_Warehouse_Allocation[21,18]",0.0
460,"Single_Warehouse_Allocation[21,19]",0.0


In [15]:
y_values.to_csv("full_solution.csv", index = False)

NameError: name 'y_values' is not defined